In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import keras.utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import keras.regularizers as kr
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import keras.losses as ls
import matplotlib.pyplot as plt
import matplotlib
import sys
import math as m
np.set_printoptions(threshold=sys.maxsize)
%matplotlib qt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

Using TensorFlow backend.


In [2]:
###Vengono caricati i dati dal file, questa volta non viene fatta nessuna selezione a priori
###a parte quella per numero di elementi non nulli.

d=pd.read_csv('./Settings/Data/Default/DEFGBData.csv')
nancount=[]
ind=[]
for i in range(0,d.shape[0]):
    nancount.append(d.iloc[i,12:].isnull().sum())
    if nancount[i]==0:
        ind.append(i)
    else:
        pass
data=d.iloc[ind,:]
data=data.set_index(['GES_FLD'])

data=data.reset_index()   
data1=data.values

dataset=data.values[:,12:].astype(float)
datafei=data.values[:,9].astype(float)
dataset=np.nan_to_num(dataset)
datafei=np.nan_to_num(datafei)
dataset=np.c_[dataset,datafei]

asun=[6.24,7.60,6.47,6.34,3.15,3.15,4.95,3.93,5.64,5.64,7.50,7.50,4.99,6.22,2.21]
for i in range(0,dataset.shape[0]):
    for j in range(0,dataset.shape[1]-1):
        if j==10:
            pass
        else:
            dataset[i,j]=(dataset[i,j]-asun[j])-(dataset[i,10]-asun[10])

dataset=np.delete(dataset,10,1)
invar=dataset

In [48]:
###Selezione del target.
###Target disponibili: 'NGC2243', 'NGC2420', 'Rup134', 'Trumpler20', 'Trumpler23'
target='Trumpler20'
path="./Settings/Weights/target/prova"
path=path.replace('target',target)
files=['/gbl0.npy','/gbl0bias.npy','/gbl1.npy','/gbl1bias.npy','/gbl2.npy','/gbl2bias.npy','/gbl3.npy','/gbl3bias.npy']
for i in range(len(files)):
    files[i]=path+files[i]
w0=np.load(files[0])
b0=np.load(files[1])
w1=np.load(files[2])
b1=np.load(files[3])
w2=np.load(files[4])
b2=np.load(files[5])
w3=np.load(files[6])
b3=np.load(files[7])

path2="./Settings/target/prova"
path2=path2.replace('target',target)
files2=['/vallossgb.npy','/trainlossgb.npy','/valaccuracygb.npy','/valsetinputgb.npy','/valsetoutputgb.npy','/valindgb.npy']
for i in range(len(files2)):
    files2[i]=path2+files2[i]
valloss=np.load(files2[0])
loss=np.load(files2[1])
valacc=np.load(files2[2])
valin=np.load(files2[3])
valout=np.load(files2[4])
valind=np.load(files2[5])

In [49]:
###Plot combinato di loss e accuracy

y1 = np.random.random_sample(len(valacc))
y2 = np.random.normal(0.0,0.2,len(valacc))
jittery=valacc+0.1*y2
title='Training vs Validation Loss funtions & Accuracy, GB Stars, target'
title=title.replace('target',target)
plt.close()
fig, (ax1, ax2) = plt.subplots(figsize=(12, 10),nrows=2, sharex=True)
plt.subplots_adjust(wspace=0, hspace=0)
ax1.plot(loss, label='Training set')
ax1.plot(valloss, label='Validation set')
ax2.scatter(list(range(1,jittery.shape[0]+1)),jittery,marker='.',color='blue',facecolor='None',label='Validation accuracy')
ax1.title.set_text(title)
plt.xlabel('Epochs',fontsize='large')
ax1.legend(loc='upper right')
ax2.legend(loc='lower right')
plt.show()


In [50]:
###Nuova NN. Vengono caricati i files preparati con il training e associati ai rispettivi layers.

model=Sequential()
model.add(Dense(invar.shape[1],input_dim=invar.shape[1],activation='selu',kernel_regularizer=kr.l1(0.01)))
model.add(Dense(round(invar.shape[1]*2/5),activation='selu',kernel_regularizer=kr.l1(0.01)))
model.add(Dense(2,activation='softsign'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.layers[0].set_weights([w0,b0])
model.layers[1].set_weights([w1,b1])
model.layers[2].set_weights([w2,b2])

In [51]:
###Viene fatta una run con tutte le stelle, si selezionano quali visualizzare direttamente nei grafici.

out=model.predict(dataset)

In [57]:
###
obj=['NGC2243','Rup134','Trumpler20','NGC2420','Trumpler23']
while obj[0]!=target:
    obj=np.roll(obj,1)

ob1=obj[0]
ob2=obj[1]
ob3=obj[2]
ob4=obj[3]
ob5=obj[4]

import matplotlib.gridspec as gridspec
c1=0
c2=0
c3=0
c4=0
c5=0
plt.close()
gs=gridspec.GridSpec(2,2)
gs.update(wspace=0.025, hspace=0.025)
fig=plt.figure(figsize=(12,10))
ax1=fig.add_subplot(gs[0,0])
ax3=fig.add_subplot(gs[1,1])
ax2=fig.add_subplot(gs[0,1],sharex=ax3,sharey=ax1)
ax1.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=True, right=True, labelright=False)
ax2.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=False, right=True, labelright=True)
ax3.tick_params(top=True, labeltop=False, bottom=True, labelbottom=True, left=True, labelleft=False, right=True, labelright=True)
ax1.set_xlabel('Output Neuron0')
ax1.xaxis.set_label_position('top') 
ax1.set_ylabel('Output Neuron2')
ax2.set_xlabel('Output Neuron1')
ax2.xaxis.set_label_position('top')
ax2.set_ylabel('Output Neuron2')
ax2.yaxis.set_label_position('right')
ax3.set_xlabel('Output Neuron1')
ax3.set_ylabel('Output Neuron0')
ax3.yaxis.set_label_position('right')

for i in range(0,dataset.shape[0]):
    if data1[i,0]==ob1:
        if c1==0:
            ax1.scatter(out[i,0],out[i,2],color='red',marker='+',label=ob1)
            c1=1
        elif c1==1:
            ax1.scatter(out[i,0],out[i,2],color='red',marker='+')
        ax2.scatter(out[i,1],out[i,2],color='red',marker='+')
        ax3.scatter(out[i,1],out[i,0],color='red',marker='+')
    elif data1[i,0]==ob2:
        if c2==0:
            ax1.scatter(out[i,0],out[i,2],color='grey',marker='1',label=ob2)
            c2=1
        elif c2==1:
            ax1.scatter(out[i,0],out[i,2],color='grey',marker='1')
        ax2.scatter(out[i,1],out[i,2],color='grey',marker='1')
        ax3.scatter(out[i,1],out[i,0],color='grey',marker='1')
    elif data1[i,0]==ob3:
        if c3==0:
            ax1.scatter(out[i,0],out[i,2],color='grey',marker='4',label=ob3)
            c3=1
        elif c3==1:
            ax1.scatter(out[i,0],out[i,2],color='grey',marker='4')
        ax2.scatter(out[i,1],out[i,2],color='grey',marker='4')
        ax3.scatter(out[i,1],out[i,0],color='grey',marker='4')
    elif data1[i,0]==ob4:
        if c4==0:
            ax1.scatter(out[i,0],out[i,2],color='grey',marker='.',label=ob4)
            c4=1
        elif c4==1:
            ax1.scatter(out[i,0],out[i,2],color='grey',marker='.')
        ax2.scatter(out[i,1],out[i,2],color='grey',marker='.')
        ax3.scatter(out[i,1],out[i,0],color='grey',marker='.')
    elif data1[i,0]==ob5:
        if c5==0:
            ax1.scatter(out[i,0],out[i,2],color='grey',marker='x',label=ob5)
            c5=1
        elif c5==1:
            ax1.scatter(out[i,0],out[i,2],color='grey',marker='x')
        ax2.scatter(out[i,1],out[i,2],color='grey',marker='x')
        ax3.scatter(out[i,1],out[i,0],color='grey',marker='x')

fig.legend(bbox_to_anchor=(0.4,0.4), loc="upper right", borderaxespad=0, fontsize='x-large')

In [24]:
###Secondo grafico, con OC del training
ob01='NGC2420'
ob02='Rup134'
ob03='Trumpler20'
ob04='NGC2243'
ob05='Trumpler23'

import matplotlib.gridspec as gridspec
c01=0
c02=0
c03=0
c04=0
c05=0
plt.close()

gs=gridspec.GridSpec(2,2)
gs.update(wspace=0.025, hspace=0.025)
fig=plt.figure(figsize=(8,8))
fig
ax1=fig.add_subplot(gs[0,0])
ax3=fig.add_subplot(gs[1,1])
ax2=fig.add_subplot(gs[0,1],sharex=ax3,sharey=ax1)
ax1.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=True, right=True, labelright=False)
ax2.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=False, right=True, labelright=True)
ax3.tick_params(top=True, labeltop=False, bottom=True, labelbottom=True, left=True, labelleft=False, right=True, labelright=True)
ax1.set_xlabel('Output Neuron0')
ax1.xaxis.set_label_position('top') 
ax1.set_ylabel('Output Neuron2')
ax2.set_xlabel('Output Neuron1')
ax2.xaxis.set_label_position('top')
ax2.set_ylabel('Output Neuron2')
ax2.yaxis.set_label_position('right')
ax3.set_xlabel('Output Neuron1')
ax3.set_ylabel('Output Neuron0')
ax3.yaxis.set_label_position('right')

for i in range(0,dataset.shape[0]):
    if data1[i,0]==ob01:
        if c01==0:
            ax1.scatter(out[i,0],out[i,2],color='red',marker='+',label=ob01)
            c01=1
        elif c01==1:
            ax1.scatter(out[i,0],out[i,2],color='red',marker='+')
        ax2.scatter(out[i,1],out[i,2],color='red',marker='+')
        ax3.scatter(out[i,1],out[i,0],color='red',marker='+')
    elif data1[i,0]==ob02:
        if c02==0:
            ax1.scatter(out[i,0],out[i,2],color='blue',marker='1',label=ob02)
            c02=1
        elif c02==1:
            ax1.scatter(out[i,0],out[i,2],color='blue',marker='1')
        ax2.scatter(out[i,1],out[i,2],color='blue',marker='1')
        ax3.scatter(out[i,1],out[i,0],color='blue',marker='1')
    elif data1[i,0]==ob03:
        if c03==0:
            ax1.scatter(out[i,0],out[i,2],color='green',marker='4',label=ob03)
            c03=1
        elif c03==1:
            ax1.scatter(out[i,0],out[i,2],color='green',marker='4')
        ax2.scatter(out[i,1],out[i,2],color='green',marker='4')
        ax3.scatter(out[i,1],out[i,0],color='green',marker='4')
    elif data1[i,0]==ob04:
        if c04==0:
            ax1.scatter(out[i,0],out[i,2],color='black',marker='.',label=ob04)
            c04=1
        elif c04==1:
            ax1.scatter(out[i,0],out[i,2],color='black',marker='.')
        ax2.scatter(out[i,1],out[i,2],color='black',marker='.')
        ax3.scatter(out[i,1],out[i,0],color='black',marker='.')
    elif data1[i,0]==ob05:
        if c05==0:
            ax1.scatter(out[i,0],out[i,2],color='cyan',marker='x',label=ob05)
            c05=1
        elif c05==1:
            ax1.scatter(out[i,0],out[i,2],color='cyan',marker='x')
        ax2.scatter(out[i,1],out[i,2],color='cyan',marker='x')
        ax3.scatter(out[i,1],out[i,0],color='cyan',marker='x')

fig.legend(bbox_to_anchor=(0.4,0.4), loc="upper right", borderaxespad=0, fontsize='x-large')


In [76]:
###Prova plot 3d fatta per i nuovi OC

plt.close()
from mpl_toolkits.mplot3d import Axes3D
fig=plt.figure()
ax=fig.add_subplot(111,projection='3d')
for i in range(0,dataset.shape[0]):
    if data1[i,0]==ob1:
        ax.scatter(out[i,0],out[i,1],out[i,2],color='red',marker='+')
    elif data1[i,0]==ob2:
        ax.scatter(out[i,0],out[i,1],out[i,2],color='grey',marker='1')
    elif data1[i,0]==ob3:
        ax.scatter(out[i,0],out[i,1],out[i,2],color='grey',marker='4')
    elif data1[i,0]==ob4:
        ax.scatter(out[i,0],out[i,1],out[i,2],color='grey',marker='.')
    elif data1[i,0]==ob5:
        ax.scatter(out[i,0],out[i,1],out[i,2],color='grey',marker='x')

In [52]:
dummyout=np_utils.to_categorical(valout)
model2=Sequential()
model2.add(Dense(invar.shape[1],input_dim=invar.shape[1],activation='selu',kernel_regularizer=kr.l1(0.01)))
model2.add(Dense(round(invar.shape[1]*2/5),activation='selu',kernel_regularizer=kr.l1(0.01)))
model2.add(Dense(2,activation='softsign'))
model2.add(Dense(dummyout.shape[1],activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model2.layers[0].set_weights([w0,b0])
model2.layers[1].set_weights([w1,b1])
model2.layers[2].set_weights([w2,b2])
model2.layers[3].set_weights([w3,b3])

out2=model2.predict(valin)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dummyout.argmax(axis=1),out2.argmax(axis=1))
prec=cm[1,1]/(cm[1,1]+cm[0,1])
recall=cm[1,1]/(cm[1,1]+cm[1,0])
accur=(cm[1,1]+cm[0,0])/(cm[0,0]+cm[0,1]+cm[1,0]+cm[1,1])
fscore=cm[1,1]/(cm[1,1]+0.5*(cm[1,0]+cm[0,1]))
print('Target: ',target)
print('TP = ',cm[1,1],', TN = ',cm[0,0],', FP = ',cm[0,1],', FN = ',cm[1,0])
print('Precision = ',prec,', Recall = ',recall,', Accuracy = ',accur)
print('F-score = ',fscore)


Target:  Trumpler20
TP =  20 , TN =  75 , FP =  3 , FN =  0
Precision =  0.8695652173913043 , Recall =  1.0 , Accuracy =  0.9693877551020408
F-score =  0.9302325581395349


In [38]:
###Plot del validation set
import matplotlib.gridspec as gridspec
title='Validation set, target'
title=title.replace('target',target)
plt.close()
gs=gridspec.GridSpec(2,2)
gs.update(wspace=0.025, hspace=0.025)
fig=plt.figure(figsize=(12,10))
fig.suptitle(title,fontsize=18)
ax1=fig.add_subplot(gs[0,0])
ax3=fig.add_subplot(gs[1,1])
ax2=fig.add_subplot(gs[0,1],sharex=ax3,sharey=ax1)
ax1.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=True, right=True, labelright=False)
ax2.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=False, right=True, labelright=True)
ax3.tick_params(top=True, labeltop=False, bottom=True, labelbottom=True, left=True, labelleft=False, right=True, labelright=True)
ax1.set_xlabel('Output Neuron0')
ax1.xaxis.set_label_position('top') 
ax1.set_ylabel('Output Neuron2')
ax2.set_xlabel('Output Neuron1')
ax2.xaxis.set_label_position('top')
ax2.set_ylabel('Output Neuron2')
ax2.yaxis.set_label_position('right')
ax3.set_xlabel('Output Neuron1')
ax3.set_ylabel('Output Neuron0')
ax3.yaxis.set_label_position('right')

legend_elements = [Line2D([0],[0],marker='.',color='none',markeredgecolor='red',markerfacecolor='red',label='TruePos'),
                   Line2D([0],[0],marker='D',color='none',markeredgecolor='red',markerfacecolor='none',label='FalseNeg'),
                   Line2D([0],[0],marker='.',color='none',markeredgecolor='grey',markerfacecolor='grey',label='TrueNeg'),
                   Line2D([0],[0],marker='D',color='none',markeredgecolor='grey',markerfacecolor='none',label='FalsePos')]

for i in range(0,dataset.shape[0]):
    if i in valind and data1[i,0]==target and out2.argmax(axis=1)[np.where(valind==i)]==1.:
        ax1.scatter(out[i,0],out[i,2],color='red',marker='.')
        ax2.scatter(out[i,1],out[i,2],color='red',marker='.')
        ax3.scatter(out[i,1],out[i,0],color='red',marker='.')
    elif i in valind and data1[i,0]==target and out2.argmax(axis=1)[np.where(valind==i)]==0.:
        ax1.scatter(out[i,0],out[i,2],color='red',marker='D',facecolor='none')
        ax2.scatter(out[i,1],out[i,2],color='red',marker='D',facecolor='none')
        ax3.scatter(out[i,1],out[i,0],color='red',marker='D',facecolor='none')
    elif i in valind and data1[i,0]!=target and out2.argmax(axis=1)[np.where(valind==i)]==0.:
        ax1.scatter(out[i,0],out[i,2],color='grey',marker='.')
        ax2.scatter(out[i,1],out[i,2],color='grey',marker='.')
        ax3.scatter(out[i,1],out[i,0],color='grey',marker='.')
    elif i in valind and data1[i,0]!=target and out2.argmax(axis=1)[np.where(valind==i)]==1.:
        ax1.scatter(out[i,0],out[i,2],color='grey',marker='D',facecolor='none')
        ax2.scatter(out[i,1],out[i,2],color='grey',marker='D',facecolor='none')
        ax3.scatter(out[i,1],out[i,0],color='grey',marker='D',facecolor='none')
    else:
        pass
    
fig.legend(handles=legend_elements,bbox_to_anchor=(0.45,0.45), loc="upper right", borderaxespad=0, fontsize='x-large')
fig.text(0.41,0.25,'Precision = ',fontsize=13,ha='right')
fig.text(0.41,0.25,round(prec,2),fontsize=13)
fig.text(0.41,0.22,'Recall = ',fontsize=13,ha='right')
fig.text(0.41,0.22,round(recall,2),fontsize=13)
fig.text(0.41,0.19,'Accuracy = ',fontsize=13,ha='right')
fig.text(0.41,0.19,round(accur,2),fontsize=13)
fig.text(0.41,0.16,'F-score = ',fontsize=13,ha='right')
fig.text(0.41,0.16,round(fscore,2),fontsize=13)

Text(0.41, 0.16, '0.57')

In [21]:
import matplotlib.gridspec as gridspec
plt.close()
gs=gridspec.GridSpec(2,2)
gs.update(wspace=0.025, hspace=0.025)
fig=plt.figure(figsize=(12,10))
ax1=fig.add_subplot(gs[0,0])
ax3=fig.add_subplot(gs[1,1])
ax2=fig.add_subplot(gs[0,1],sharex=ax3,sharey=ax1)
ax1.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=True, right=True, labelright=False)
ax2.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=False, right=True, labelright=True)
ax3.tick_params(top=True, labeltop=False, bottom=True, labelbottom=True, left=True, labelleft=False, right=True, labelright=True)
ax1.set_xlabel('Output Neuron0')
ax1.xaxis.set_label_position('top') 
ax1.set_ylabel('Output Neuron2')
ax2.set_xlabel('Output Neuron1')
ax2.xaxis.set_label_position('top')
ax2.set_ylabel('Output Neuron2')
ax2.yaxis.set_label_position('right')
ax3.set_xlabel('Output Neuron1')
ax3.set_ylabel('Output Neuron0')
ax3.yaxis.set_label_position('right')

for i in range(0,dataset.shape[0]):
    if i in valind and data1[i,0]==target:
        ax1.scatter(out[i,0],out[i,2],color='red',marker='D',facecolor='none')
        ax2.scatter(out[i,1],out[i,2],color='red',marker='D',facecolor='none')
        ax3.scatter(out[i,1],out[i,0],color='red',marker='D',facecolor='none')
    elif i not in valind and data1[i,0]==target:
        ax1.scatter(out[i,0],out[i,2],color='red',marker='.')
        ax2.scatter(out[i,1],out[i,2],color='red',marker='.')
        ax3.scatter(out[i,1],out[i,0],color='red',marker='.')
    elif i in valind and data1[i,0]!=target:
        ax1.scatter(out[i,0],out[i,2],color='grey',marker='D',facecolor='none')
        ax2.scatter(out[i,1],out[i,2],color='grey',marker='D',facecolor='none')
        ax3.scatter(out[i,1],out[i,0],color='grey',marker='D',facecolor='none')
    elif i not in valind and data1[i,0]!=target:
        ax1.scatter(out[i,0],out[i,2],color='grey',marker='.')
        ax2.scatter(out[i,1],out[i,2],color='grey',marker='.')
        ax3.scatter(out[i,1],out[i,0],color='grey',marker='.')
        

In [53]:
###Plot del validation set
import matplotlib.gridspec as gridspec
title='Validation set, target'
title=title.replace('target',target)
plt.close()
plt.figure(figsize=(12,10))
plt.title(title,fontsize=18)

legend_elements = [Line2D([0],[0],marker='.',color='none',markeredgecolor='red',markerfacecolor='red',label='TruePos'),
                   Line2D([0],[0],marker='D',color='none',markeredgecolor='red',markerfacecolor='none',label='FalseNeg'),
                   Line2D([0],[0],marker='.',color='none',markeredgecolor='grey',markerfacecolor='grey',label='TrueNeg'),
                   Line2D([0],[0],marker='D',color='none',markeredgecolor='grey',markerfacecolor='none',label='FalsePos')]

for i in range(0,dataset.shape[0]):
    if i in valind and data1[i,0]==target and out2.argmax(axis=1)[np.where(valind==i)]==1.:
        plt.scatter(out[i,0],out[i,1],color='red',marker='.')
    elif i in valind and data1[i,0]==target and out2.argmax(axis=1)[np.where(valind==i)]==0.:
        plt.scatter(out[i,0],out[i,1],color='red',marker='D',facecolor='none')
    elif i in valind and data1[i,0]!=target and out2.argmax(axis=1)[np.where(valind==i)]==0.:
        plt.scatter(out[i,0],out[i,1],color='grey',marker='.')
    elif i in valind and data1[i,0]!=target and out2.argmax(axis=1)[np.where(valind==i)]==1.:
        plt.scatter(out[i,0],out[i,1],color='grey',marker='D',facecolor='none')
    else:
        pass
    
plt.legend(handles=legend_elements,bbox_to_anchor=(0.05,0.95), loc="upper left", borderaxespad=0, fontsize='x-large')
plt.text(-0.42,0.25,'Precision = ',fontsize=13,ha='right')
plt.text(-0.42,0.25,round(prec,2),fontsize=13)
plt.text(-0.42,0.22,'Recall = ',fontsize=13,ha='right')
plt.text(-0.42,0.22,round(recall,2),fontsize=13)
plt.text(-0.42,0.19,'Accuracy = ',fontsize=13,ha='right')
plt.text(-0.42,0.19,round(accur,2),fontsize=13)
plt.text(-0.42,0.16,'F-score = ',fontsize=13,ha='right')
plt.text(-0.42,0.16,round(fscore,2),fontsize=13)

Text(-0.42, 0.16, '0.93')